In [ ]:
import json

# Data to write
kaggle_credentials = {
    "username": "harikaran123r",
    "key": "0e4de6becd2e7a91773fd195c8878f3a"
}

# Write to kaggle.json
with open("kaggle.json", "w") as json_file:
    json.dump(kaggle_credentials, json_file, indent=4)

print("kaggle.json file created successfully.")

import os
import zipfile

# Create the kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)

# Move kaggle.json into the directory
!mv kaggle.json /root/.kaggle/

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json

# Download and unzip the dataset
!pip install kaggle
!kaggle datasets download -d wasiqaliyasir/breast-cancer-dataset -p /content/data --unzip
print("Dataset downloaded and unzipped successfully.")

kaggle.json file created successfully.
Dataset URL: https://www.kaggle.com/datasets/wasiqaliyasir/breast-cancer-dataset
License(s): apache-2.0
  0% 0.00/48.7k [00:00<?, ?B/s]
100% 48.7k/48.7k [00:00<00:00, 169MB/s]
Dataset downloaded and unzipped successfully.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.impute import KNNImputer

# Load CSV (assuming processed by previous steps)
data_dir = "/content/data"
os.makedirs(data_dir, exist_ok=True)
labels_df = pd.read_csv(f"{data_dir}/Breast_cancer_dataset.csv").dropna().reset_index(drop=True)
print(f"Original CSV entries: {len(labels_df)}")
if len(labels_df) == 0:
    print("Warning: Dataset is empty. Please verify the file or download.")
    exit()

# Technique: Impute missing values using KNN (K-Nearest Neighbors) imputation.
# Justification: While this dataset has no missing values, KNN imputation can estimate missing feature values (e.g., 'radius_mean') based on similar samples, making the pipeline robust for future datasets with incomplete tumor measurements, enhancing cancer diagnosis reliability.
# Implementation
# Create a copy to simulate potential missing data (for demonstration)
labels_df_impute = labels_df.copy()
if len(labels_df_impute) == 0:
    print("Warning: No data to impute. Skipping imputation step.")
else:
    # Simulate missing values (e.g., 5% of 'radius_mean' set to NaN)
    np.random.seed(42)
    mask = np.random.random(labels_df_impute['radius_mean'].shape) < 0.05
    labels_df_impute.loc[mask, 'radius_mean'] = np.nan
    print(f"Number of simulated missing values in 'radius_mean': {labels_df_impute['radius_mean'].isnull().sum()}")

    # Apply KNN imputation
    imputer = KNNImputer(n_neighbors=5)
    features = labels_df_impute.drop(['id', 'diagnosis'], axis=1).columns
    labels_df_impute[features] = imputer.fit_transform(labels_df_impute[features])
    print(f"Missing values after imputation: {labels_df_impute['radius_mean'].isnull().sum()}")

    # EDA Visualization: Histogram of 'radius_mean' before and after imputation
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.hist(labels_df['radius_mean'], bins=20, alpha=0.7, label='Original')
    plt.hist(labels_df_impute.loc[mask, 'radius_mean'], bins=20, alpha=0.5, label='Missing (Before)', color='red')
    plt.title('Radius Mean Before Imputation')
    plt.ylabel('Frequency')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.hist(labels_df_impute['radius_mean'], bins=20, alpha=0.7, label='Imputed')
    plt.title('Radius Mean After Imputation')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()
    print('Interpretation: Imputation smoothly integrates missing values into the distribution, maintaining data integrity for modeling.')

Original CSV entries: 0
